<a href="https://colab.research.google.com/github/budding-tech-savvy/-Optimizing-Last-Mile-Delivery-Through-Geospatial-Clustering-of-Nearby-Societies-/blob/main/clustering_societies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
file_path = "/content/drive/MyDrive/Reduce vehicle  count and grouping societies/latitude and longitude data.csv"  # Replace with your file path

In [20]:
data = pd.read_csv(file_path,sep="\t")

In [21]:
print(data)

             Society Name   Latitude  Longitude
0      Greenfield Heights  28.704131  77.053478
1             Emerald Bay  28.738793  77.071840
2     Pinewood Residences  28.703083  77.108263
3    Moonlight Apartments  28.711415  77.078376
4       Sunny Side Villas  28.703065  77.099209
..                    ...        ...        ...
245        Crystal Towers  12.937430  77.545046
246        Golden Springs  12.932398  77.617364
247      Rosewood Society  12.932283  77.544695
248        Crystal Towers  13.009791  77.619113
249      Lakeview Enclave  12.941229  77.644414

[250 rows x 3 columns]


In [22]:
coords = data[['Latitude', 'Longitude']].to_numpy()

In [23]:
from geopy.distance import great_circle

In [28]:
kms_per_radian = 6371.0088  # Earth's radius in km
epsilon = 3 / kms_per_radian  # Set clustering radius to 3 km
db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

In [25]:
data['Cluster'] = db.labels_

In [26]:
print(data)

             Society Name   Latitude  Longitude  Cluster
0      Greenfield Heights  28.704131  77.053478        0
1             Emerald Bay  28.738793  77.071840        0
2     Pinewood Residences  28.703083  77.108263        0
3    Moonlight Apartments  28.711415  77.078376        0
4       Sunny Side Villas  28.703065  77.099209        0
..                    ...        ...        ...      ...
245        Crystal Towers  12.937430  77.545046        4
246        Golden Springs  12.932398  77.617364        4
247      Rosewood Society  12.932283  77.544695        4
248        Crystal Towers  13.009791  77.619113        4
249      Lakeview Enclave  12.941229  77.644414        4

[250 rows x 4 columns]


In [29]:
output_path = "clustered_data.xlsx"
data.to_excel(output_path, index=False)
print(f"Clustered data saved to {output_path}")

Clustered data saved to clustered_data.xlsx


In [ ]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

# Create a base map centered at the mean latitude and longitude
center_lat = data['Latitude'].mean()
center_lon = data['Longitude'].mean()
base_map = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add marker clusters to the map
marker_cluster = MarkerCluster().add_to(base_map)

# Add markers for each society with cluster info
for _, row in data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Society: {row['Society Name']}<br>Cluster: {row['Cluster']}",
        icon=folium.Icon(color="blue" if row['Cluster'] != -1 else "red")
    ).add_to(marker_cluster)

# Display the map
base_map

